# VideoProcessing

In [ ]:
import os
import random
import numpy as np
import cv2
import torch
from torchvision import models, transforms


def video_to_frames(video_path, temp_path, frames_step):
    # Extract frames from a video using OpenCV
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0
    frames = []

    while success:
        frame_path = os.path.join(temp_path, f"frame{count}.jpg")
        cv2.imwrite(frame_path, image)  # save frame as JPEG file
        success, image = vidcap.read()
        count += 1

        if count % frames_step == 0:
            # Read the last frames_step frames and convert them to tensor
            last_frames = []
            for i in range(frames_step):
                frame_path = os.path.join(temp_path, f"frame{count - frames_step + i}.jpg")
                frame = cv2.imread(frame_path)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                last_frames.append(frame)

            # Apply transforms and add to frames list
            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
            last_frames = [transform(frame).numpy() for frame in last_frames]
            frames.extend(last_frames)

    return frames




def extract_features(frames):
    # Extract features using a pre-trained ResNet50 model
    model = models.resnet50(pretrained=True)
    model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove last FC layer
    model.eval()

    with torch.no_grad():
        features = []
        frames_count = len(frames)
        
        # Generate a list of indices from 0 to frames_count-1
        indices = list(range(frames_count))
        # Shuffle the indices randomly
        random.shuffle(indices)
        # Take the first 80 indices
        selected_indices = indices[:80]
        # Sort the selected indices in ascending order
        selected_indices.sort()
        
        for index in selected_indices:
            frame = frames[index]

            frame_tensor = torch.Tensor(frame).unsqueeze(0)
            feature = model(frame_tensor).squeeze().numpy()
            features.append(feature)

    features = np.array(features)
    features = np.resize(features, (min(80, frames_count), 4096))
    return features



def video_to_features(video_dir, temp_dir, feature_dir, frames_step=80):
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(feature_dir, exist_ok=True)

    for video_name in os.listdir(video_dir):
        video_path = os.path.join(video_dir, video_name)
        video_name = os.path.splitext(video_name)[0]
        temp_path = os.path.join(temp_dir, video_name)
        os.makedirs(temp_path, exist_ok=True)
        feature_path = os.path.join(feature_dir, f"{video_name}.npy")

        frames = video_to_frames(video_path, temp_path, frames_step)
        if frames:
            features = extract_features(frames)
            np.save(feature_path, features)

        # Clean up temporary frames
        for frame_name in os.listdir(temp_path):
            frame_path = os.path.join(temp_path, frame_name)
            os.remove(frame_path)
        os.rmdir(temp_path)


: 

In [2]:
video_dir = "C:/Users/User/Documents/P2M/test1/YouTubeClips"
temp_dir = "C:/Users/User/Documents/P2M/test1/temp"
feature_dir = "C:/Users/User/Documents/P2M/test1/feature"

In [3]:
video_to_features(video_dir,temp_dir,feature_dir)

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python38\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
